In [ ]:
import numpy as np
import random

def sigmoid(z): # сигмоидальная (логистическая) функция
    return 1.0/(1.0+np.exp(-z))

In [ ]:
# класс нейронной сети

class Network:

    def __init__(self, sizes, l1=0, l2=0): # инициализация нейронной сети
        # len(size) - кол-во слоев нейронной сети, i-ый элемент списка sizes - кол-во нейронов на i+1 слое
        self.num_layers = len(sizes) # задаем к-во слоев
        self.sizes = sizes # задаем к-во нейронов на каждом слое
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] # задаем начальный вектор смещения весов
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] # задаем начальные веса
        self.l1 = l1 # коэффициент для регуляризации L1
        self.l2 = l2 # коэффициент для регуляризации L2

        
    def SGD(self, X, y, epochs, batch_size, learning_rate, eps):
        # обучение нейронной сети методом стохастического градиентного спуска
        error = True
        i = 0
        while error and i < epochs:
            i += 1
            error = False
            indexes = list(range(len(X)))
            sample_ind = np.random.choice(indexes, batch_size)
            X_example = X[sample_ind]
            y_answer = y[sample_ind]
            self.update_mini_batch(X_example, y_answer, learning_rate)
            error2 = self.error_function(X, y)
            if error2 > eps:
                error = True
        return i, error, error2
       
        
    def update_mini_batch(self, X_example, y_answer, learning_rate):
        # один шаг градиентного спуска (т.е. обновление весов) по алгоритму обратного распределения ошибки  
        # с возможностью использования регуляризации (по умолчанию коэффициенты регуляризации равны 0)
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in zip(X_example, y_answer):
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        learning_rate_2 = learning_rate / len(X_example)
        self.weights = [w - learning_rate_2 * nw - self.l1 * np.sign(w) - self.l2 * w for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - learning_rate_2 * nb for b, nb in zip(self.biases,  nabla_b)]

        
    def backprop(self, x, y):
        # вычисление градиента целевой функции
        nabla_b_2 = [np.zeros(b.shape) for b in self.biases]
        nabla_w_2 = [np.zeros(w.shape) for w in self.weights]
        # прямое распределение: расчет активаций нейронов
        a = []
        z = x
        a.append(z)
        for b, w in zip(self.biases, self.weights):
            z = sigmoid(np.dot(w, z) + b)
            a.append(z)            
        # обратное распространение
        # для выходного слоя
        delta = (a[-1] - y) * a[-1] * (1 - a[-1]) 
        nabla_b_2[-1] = delta
        nabla_w_2[-1] = delta.dot(a[-2].T)
        # для остальных слоев
        for l in range(2, self.num_layers):
            delta = self.weights[-l+1].T.dot(delta) * a[-l] * (1-a[-l])
            nabla_b_2[-l] = delta
            nabla_w_2[-l] = delta.dot(a[-l-1].T)
        return nabla_b_2, nabla_w_2
    
    
    def error_function(self, X, y): # расчет значения функции ошибки
        c = 0
        for x, y_1 in zip(X, y):
            a = x
            for b, w in zip(self.biases, self.weights):
                a = sigmoid(np.dot(w, a) + b)
                y_answer = a
                c += np.sum((y_1 - y_answer)**2)
        return c / (2*len(X))

In [ ]:
# здесь будет реализован алгоритм считывания данных (после подключения источников)

# X = 
# y = 

In [ ]:
"""разбиваем код на тестовую и обучающую выборки (желательный, но не обязательный шаг, использовать если достаточно данных)
данное разбиение необходимо для сравнения качества работы нейронной сети при различных исходных данных и
выбора оптимальной модели """

test_index = np.random.choice([True, False], len(X), replace=True, p=[0.25, 0.75])
X_test  = X[test_index]
y_test  = y[test_index]
X_train = X[np.logical_not(test_index)]
y_train = y[np.logical_not(test_index)]

In [ ]:
# функция расчета ошибки на тестовой выборке

def error_function_for_test(X_test, y_test):
        c = 0
        for x, y_1 in zip(X_test, y_test):
            a = x
            for b, w in zip(n.biases, n.weights):
                a = sigmoid(np.dot(w, a) + b)
                y_answer = a
                c += np.sum((y_1 - y_answer)**2)
        return c / (2*len(X)) 

In [ ]:
# запуск нейронной сети если есть тестовая выборка

size = [2, 4, 5] # задаем "размерность" нейронной сети - к-во слоев и к-во нейронов на каждом слое

"""инициализируем нейронную сеть, два последних параметра не обязательны, они задают коэффициенты регуляризации, 
по умолчания равны 0"""
n = Network(size, 1, 1) 

B_start = n.biases # фиксируем стартовую точку градиентного спуска
W_start = n.weights

i, error, error2 = n.SGD(X_train, y_train, epochs=100, batch_size=2, learning_rate=1, eps=0.01) # обучаем нейронную сеть
print (i, error, error2)

B = n.biases # фиксируем результаты работы нейронной сети
W = n.weights

error3 = error_function_for_test(X_test, y_test) # тестируем эффективность нейронной сети 
print(error3)

In [ ]:
# запуск нейронной сети если нет тестовой выборки

size = [3, 2, 1] # задаем "размерность" нейронной сети - к-во слоев и к-во нейронов на каждом слое

"""инициализируем нейронную сеть, два последних параметра не обязательны, они задают коэффициенты регуляризации, 
по умолчания равны 0"""

n = Network(size, 1, 1) 

B_start = n.biases # фиксируем стартовую точку градиентного спуска
W_start = n.weights

i, error, error2 = n.SGD(X, y, epochs=10000, batch_size=100, learning_rate=1, eps=0.01) # обучаем нейронную сеть
print (i, error, error2)

B = n.biases # фиксируем результаты работы нейронной сети
W = n.weights